In [89]:
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium


In [78]:
#get data from ref
eng = mariadb_connection()
power_df = pd.read_sql(
    """SELECT p.windturbine_id, windspeed, power, windfarm_id,latitude,longitude
        FROM mariadb_itw.windturbines as wt inner join mariadb_itw.powercurves as p
        on p.windturbine_id = wt.windturbine_id ;""",
    con=eng
    )

turbines_df = pd.read_sql(
    """SELECT windturbine_id, windfarm_id, latitude, longitude
        FROM mariadb_itw.windturbines ;""",
    con=eng
    )


In [88]:

#create list of colors to match with farms
colors= ["red", "blue", "green", "purple", "orange", "darkred","lightred", "beige", "darkblue", 
         "darkgreen", "cadetblue", "darkpurple", "white", "pink", 
         "lightblue", "lightgreen", "gray", "black", "lightgray"]

#create a number from O to number of farms_id
turbines_df['code'] = pd.factorize(turbines_df['windfarm_id'])[0] 


# Create a map centered at the given latitude and longitude
map = folium.Map(location=[turbines_df.latitude.mean(), turbines_df.longitude.mean()], zoom_start=5)

for i,row in turbines_df.iterrows():

    #Setup the content of the popup
    html = f'''        
        <p style="color:blue">Farm_id: {row["windfarm_id"]}<p/>
        <p style="color:green">Turbine_id: {row["windturbine_id"]}<p/>
    '''
    iframe = folium.IFrame(html, width=300, height=150)
  
    
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300,max_heigth=100)

    try:
        icon_color = colors[row['code']]
    except:
        #Catch nans
        icon_color = 'gray'
    
    #Add each row to the map
    folium.Marker(location=[row['latitude'],row['longitude']],
                  popup = popup, 
                  icon=folium.Icon(color=icon_color, icon='')).add_to(map)


map

In [90]:

farm='1ec6d7a1-4bf8-62b0-ac1e-df2d2b033685'
p_df = power_df[power_df.windfarm_id==farm]

fig = px.line(p_df, x="windspeed", y="power", color='windturbine_id')
fig.update_layout(title_text=f"Powercurves for windfarm_id: {farm}", title_x=0.5)
fig.show()